In [12]:
import networkx as nx
import matplotlib.pyplot as plt
import pickle 
import pandas as pd
import plotly.graph_objs as pgo


In [9]:
################################################
#
# H U M A N 
#
################################################

organism = 'Human'

G = nx.read_edgelist('input/ppi_elist.txt',data=False)
# d_ent_sym, d_sym_ent = genent2sym()

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

#df_gene_sym = pd.read_csv('_output_csv/DF_gene_symbol_Human.csv', index_col=0)
#l_features = list((df_gene_sym.to_dict()).values())

posG_entrez = []
for k in G.nodes():
    posG_entrez.append(k)
    
# Gene Symbols 
df_gene_sym = pd.read_csv('_output_csv/DF_gene_symbol_Human.csv', index_col=0)
l_features = list(df_gene_sym['0'])

In [10]:
print('Number of nodes i.e. genes: %s' %len(list(G.nodes())))
print('Number of edges: %s'%len(list(G.edges())))
print('Network density: %.1f%%' %(200.*len(list(G.edges()))/(len(list(G.nodes()))*len(list(G.nodes()))-1)))

Number of nodes i.e. genes: 16376
Number of edges: 309355
Network density: 0.2%


### Spring Layout 2D

In [ ]:
%%time 

# ---------------------------------------
# Spring
# ---------------------------------------
#forceatlas2 = ForceAtlas2()
#posG_spring = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=200)
posG_spring2D = nx.spring_layout(G, dim = 2)

plt.figure(figsize=(25,25),dpi=300)

nx.draw_networkx_nodes(G, posG_spring2D, edgecolors = 'k', linewidths = 0.1, node_color='dimgrey', 
                       node_size = 30.0, 
                       alpha = 0.4)
nx.draw_networkx_edges(G, posG_spring2D, width = 0.5, edge_color = 'grey', alpha = 0.3)
plt.box(False)

plt.savefig('output_plots/spring.png')

plt.show()

### Spring Layout 3D

In [ ]:
%%time 

num_iterate = 10 

posG_spring3D  = nx.spring_layout(G, iterations = num_iterate, dim = 3)

In [ ]:
# normalize coordinates 
x_list = []
y_list = []
z_list = []
for k,v in posG_spring3D.items():
    x_list.append(v[0])
    y_list.append(v[1])
    z_list.append(v[2])
    
xx_norm = sklearn.preprocessing.minmax_scale(x_list, feature_range=(0, 1), axis=0, copy=True)
yy_norm = sklearn.preprocessing.minmax_scale(y_list, feature_range=(0, 1), axis=0, copy=True)
zz_norm = sklearn.preprocessing.minmax_scale(z_list, feature_range=(0, 1), axis=0, copy=True)

posG_spring3D_norm = dict(zip(list(G.nodes()), zip(xx_norm,yy_norm, zz_norm)))


spring_nodes = get_trace_nodes_3D(posG_spring3D_norm, None, colours, size3d)
#spring_edges = get_trace_edges_3D(G, posG_spring3D_norm, edge_colordark, opac=1)
spring_edges = get_trace_spec_edges_only(list(d_edge_col.keys()), posG_spring3D_norm, disease_col) #go_col)


spring_data = [spring_nodes, spring_edges]

print('Spring Layout:', organism)

fig = pgo.Figure()
for i in spring_data:
    fig.add_trace(i)
'''fig.update_layout(template='plotly_dark', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='black')),
                    dragmode="turntable",
                    #annotations=annotations,
                ))'''

fig.update_layout(template='plotly_white', showlegend=False, width=1200, height=1200,
                  scene=dict(
                      xaxis_title='',
                      yaxis_title='',
                      zaxis_title='',
                      xaxis=dict(nticks=0,tickfont=dict(
                            color='white')),
                      yaxis=dict(nticks=0,tickfont=dict(
                            color='white')),
                      zaxis=dict(nticks=0,tickfont=dict(
                            color='white')),    
                    dragmode="turntable",
                    #annotations=annotations,
                ))    
#py.iplot(fig)

plotly.offline.plot(fig, filename = 'output_plots/SPRING3D_'+'iter'+str(num_iterate)+'_'+organism+'.html', auto_open=True)